In [ ]:
#| default_exp noise

In [ ]:
#| include: false
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
from qsample.circuit import Circuit, GATES, unpack

Some common error sets

In [ ]:
#| export
DEPOLAR1 = {"X", "Y", "Z"}
DEPOLAR2 = {('I', 'X'), ('I', 'Y'), ('I', 'Z'),
            ('X', 'I'), ('X', 'X'), ('X', 'Y'), ('X', 'Z'),
            ('Y', 'I'), ('Y', 'X'), ('Y', 'Y'), ('Y', 'Z'),
            ('Z', 'I'), ('Z', 'X'), ('Z', 'Y'), ('Z', 'Z')}
XFLIP = {"X"}
ZFLIP = {"Z"}

In [ ]:
#| export
class ErrorModel:
    """Representation of an incoherent error model."""
    
    groups = []
    
    def group(self, circuit):
        """Must be implemented by subclass"""
        raise NotImplemented
        
    def generate(self, fgroups, circuit):
        """Must be implemented by subclass"""
        raise NotImplemented
  
    @staticmethod
    def choose_p(groups: dict, probs: list):
        """Select elements from group g_i with probability p_i."""
        return {grp: [loc for loc in locs if np.random.random() < prob] 
                for (grp,locs), prob in zip(groups.items(), probs)}
    
    @staticmethod
    def choose_w(groups: dict, weights: list):
        """Select w_i locations for group g_i in groups"""
        return {grp: [locs[i] for i in np.random.choice(len(locs), weight, replace=False)]
                for (grp,locs),weight in zip(groups.items(), weights)}
    
    def run(self, circuit, fgroups):
        """Generate new Circuit of same length as `Circuit` with faults generated
        by `self.generate` and corresponding location for each group in fgroups."""
        fault_circuit = Circuit([{} for _ in range(circuit.n_ticks)])
        
        for (tidx, qb), fop in self.generate(fgroups, circuit):
            if isinstance(qb, tuple):
                for q,op in zip(qb,fop):
                    if op == "I": continue
                    qbs = fault_circuit[tidx].get(op,set())
                    qbs.add(q)
                    fault_circuit[tidx][op] = qbs
            elif isinstance(qb, int):
                qbs = fault_circuit[tidx].get(fop,set())
                qbs.add(qb)
                fault_circuit[tidx][fop] = qbs
        return fault_circuit 

In [ ]:
#| export
class E0(ErrorModel):
    """No-Error error model"""
    
    groups = ["0"]
    
    def group(self, circuit):
        return {"0": {}}
    
    def run(self, *args, **kwargs):
        return None

In [ ]:
#| export
class E1(ErrorModel):
    """One prob/weight for all 1- and 2-qubit gates"""
    
    groups = ["q"]
    
    def group(self, circuit):
        gates = GATES["q1"] | GATES["q2"]
        return {"q": [(ti,q) for ti,t in enumerate(circuit) for g,qs in t.items() for q in qs if g in gates]}
    
    def generate(self, fgroups, circuit):
        
        errsets = {
            "q1": DEPOLAR1,
            "q2": DEPOLAR2
        }
                
        for grp, locs in fgroups.items():
            for loc in locs:
                if isinstance(loc[1], tuple):
                    errset = errsets["q2"]
                else:
                    errset = errsets["q1"]
                fop = list(errset)[np.random.choice(len(errset))]
                yield loc, fop

In [ ]:
#| export
class E2(ErrorModel):
    """Individual errors on 1-qubit and 2-qubit gates."""
    
    groups = ["q1", "q2"]
    
    def group(self, circuit):
        gates = {"q1": GATES["q1"], "q2": GATES["q2"]}
        return {grp: [(ti,q) for ti,t in enumerate(circuit) for g,qs in t.items() 
                      for q in qs if g in gset] for grp,gset in gates.items()}
    
    def generate(self, fgroups, circuit):
        errsets = {
            "q1": DEPOLAR1,
            "q2": DEPOLAR2
        }
        
        for grp, locs in fgroups.items():
            for loc in locs:
                if isinstance(loc[1], tuple):
                    errset = errsets["q2"]
                else:
                    errset = errsets["q1"]
                fop = list(errset)[np.random.choice(len(errset))]
                yield loc, fop

In [ ]:
#| export
class E3(ErrorModel):
    """Errors on all gates individual + idle."""
    
    groups = ["q1", "q2", "meas", "idle", "init"]
    
    def group(self, circuit):
        groups =  {grp: [(ti,q) for ti,t in enumerate(circuit) for g,qs in t.items() 
                         for q in qs if g in gset] for grp,gset in GATES.items()}
        qbs = set(unpack(circuit))
        groups['idle'] = [(ti,q) for ti,t in enumerate(circuit) for q in qbs.difference(set(unpack(t)))]
        return groups
    
    def generate(self, fgroups, circuit):
        
        errsets = {
            'q1': DEPOLAR1,
            'q2': DEPOLAR2,
            'meas': XFLIP,
            'idle': ZFLIP,
            'init': XFLIP   
        }
        
        for grp, locs in fgroups.items():
            for loc in locs:
                fop = list(errsets[grp])[np.random.choice(len(errsets[grp]))]
                yield loc, fop

In [ ]:
#| export
class E3_1(ErrorModel):
    """Like E3, but idle locations split in two subsets."""
    
    groups = ["q1", "q2", "meas", "idle1", "idle2", "init"]
    
    def group(self, circuit):
        groups =  {i: [(ti,q) for ti,t in enumerate(circuit) for g,qs in t.items() 
                          for q in qs if g in gset] for i,gset in GATES.items()}
        qbs = set(unpack(circuit))
        groups['idle1'] = [(ti,q) for ti,t in enumerate(circuit) 
                           for q in qbs.difference(set(unpack(t))) 
                           if not any([op in GATES['q2'] for op in t.keys()])]
        groups['idle2'] = [(ti,q) for ti,t in enumerate(circuit) 
                           for q in qbs.difference(set(unpack(t))) 
                           if any([op in GATES['q2'] for op in t.keys()])]
        return groups
    
    def generate(self, fgroups, circuit):
        
        errsets = {
            'q1': DEPOLAR1,
            'q2': DEPOLAR2,
            'meas': XFLIP,
            'idle1': DEPOLAR1,
            'idle2': DEPOLAR1,
            'init': XFLIP   
        }
        
        for grp, locs in fgroups.items():
            for loc in locs:
                fop = list(errsets[grp])[np.random.choice(len(errsets[grp]))]
                yield loc, fop

In [ ]:
#| export
class InnsbruckTrap(ErrorModel):
    """Extension of E3_1 adding crosstalk and idlem."""
    
    groups = ["q1", "q2", "init", "meas", "idle1", "idle2", "idlem", "c1", "c2_com", "c2_!com"]
    
    def group(self, circuit):
        res = {}
        
        # 1qb,2qb,init,meas
        noise_free = {"Z", "S", "Sd", "Rz"} # noise free gates 
        for name, gateset in GATES.items():
            res[name] = [(tidx,q) for tidx,tick in enumerate(circuit) for g,qs in tick.items() for q in qs if g in gateset and g not in noise_free]
            
        qubits = set(range(circuit.n_qubits))
            
        #idle1,idle2,idlem
        res['idle1'], res['idle2'], res['idlem'] = [], [], []
        for tidx, tick in enumerate(circuit):
            idle_qubits = [(tidx,q) for q in qubits.difference(set(unpack(tick)))]
            if any([True for g,locs in tick.items() if g == 'measure']):
                res['idlem'] += idle_qubits
            elif any([True for g,locs in tick.items() if g in GATES['q2']]):
                res['idle2'] += idle_qubits
            else:
                res['idle1'] += idle_qubits
            
        # c1,c2_(!)common
        res['c1'], res['c2_common'], res['c2_!common'] = [], [], []
        for tidx,tick in enumerate(circuit):
            
            tickqubits = set(unpack(tick))
            inactive = qubits.difference(tickqubits)

            for g,qs in tick.items():

                if g in GATES['q1']:
                    for q in qs:
                        if q-1 in inactive:
                            res['c1'].append((i,q-1))
                        elif q+1 in inactive:
                            res['c1'].append((i,q+1))

                elif g in GATES['q1']:
                    neighbors = []
                    for qtup in qs:
                        neighbor = []
                        for q in qtup:
                            if q-1 in inactive:
                                neighbor.append((i,q-1))
                            elif q+1 in inactive:
                                neighbor.append((i,q+1))
                        neighbors.append(neighbor)

                    res['c2_common'] = list(filter(lambda x: x in neighbors[1], neighbors[0]))
                    res['c2_!common'] = list(filter(lambda x: x not in neighbors[1], neighbors[0]))
        
        return res
    
    def generate(self, fgroups, circuit):
        
        errsets = {
            'q1': DEPOLAR1,
            'q2': DEPOLAR2,
            'init': XFLIP,
            'meas': XFLIP,
            'idle1': ZFLIP,
            'idle2': ZFLIP,
            'idlem': ZFLIP,  
            'c1': {},
            'c2_com': {},
            'c2_!com': {}
        }
        
        pass